In [9]:
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

model_path = "/jizhicfs/hymiezhao/models/Qwen2.5-Math-7B-16k-think"

# 初始化模型和tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
sampling_params = SamplingParams(
    temperature=1.0, 
    top_p=1.0, 
    max_tokens=8192,
    logprobs=1  # 确保返回logprobs
)
#llm = LLM(model=model_path, tensor_parallel_size=4)

In [10]:
import pandas as pd

input_file = "../dataset/sub_5000_openr1.parquet"
df = pd.read_parquet(input_file)
df.head()

,data_source,prompt,target,ability,reward_model,extra_info
13505,olympiads,[{'content': 'Your task is to follow a systema...,"[{'content': '<think> Alright, let's tackle th...",,"{'ground_truth': '0', 'style': 'rule'}","{'index': -1, 'split': 'default'}"
41680,olympiads,[{'content': 'Your task is to follow a systema...,"[{'content': '<think> Okay, so I need to find ...",,"{'ground_truth': '45', 'style': 'rule'}","{'index': -1, 'split': 'default'}"
36327,olympiads,[{'content': 'Your task is to follow a systema...,"[{'content': '<think> Okay, let's see. I need ...",,"{'ground_truth': '5', 'style': 'rule'}","{'index': -1, 'split': 'default'}"
4336,olympiads,[{'content': 'Your task is to follow a systema...,"[{'content': '<think> Alright, let's tackle th...",,"{'ground_truth': '0.75', 'style': 'rule'}","{'index': -1, 'split': 'default'}"
9035,olympiads,[{'content': 'Your task is to follow a systema...,"[{'content': '<think> Okay, let's see. So the ...",,"{'ground_truth': '34', 'style': 'rule'}","{'index': -1, 'split': 'default'}"


In [11]:
df.iloc[0].to_dict()

{'data_source': 'olympiads',
 'prompt': array([{'content': 'Your task is to follow a systematic, thorough reasoning process before providing the final solution. This involves analyzing, summarizing, exploring, reassessing, and refining your thought process through multiple iterations. Structure your response into two sections: Thought and Solution. In the Thought section, present your reasoning using the format: "<think>\n {thoughts} </think>\n". Each thought should include detailed analysis, brainstorming, verification, and refinement of ideas. After "</think>\n," in the Solution section, provide the final, logical, and accurate answer, clearly derived from the exploration in the Thought section. If applicable, include the answer in \\boxed{} for closed-form results like multiple choices or mathematical solutions.', 'role': 'system'},
        {'content': "7.177. $\\frac{\\log _{2}\\left(9-2^{x}\\right)}{3-x}=1$.\n\nTranslate the above text into English, please keep the original text's

In [17]:
sample = df.iloc[0]

message = sample['prompt']
target = sample['target'][0]

prompt_with_chat_template = tokenizer.apply_chat_template(
                message,
                tokenize=False,
                add_generation_prompt=True
            )

if prompt_with_chat_template.endswith('<think>\n') and target['content'].startswith('<think>\n'):
    target['content'] = target['content'][len('<think>\n'):]

print(prompt_with_chat_template)
print(target['content'])

Your task is to follow a systematic, thorough reasoning process before providing the final solution. This involves analyzing, summarizing, exploring, reassessing, and refining your thought process through multiple iterations. Structure your response into two sections: Thought and Solution. In the Thought section, present your reasoning using the format: “<think>
 {thoughts} </think>
”. Each thought should include detailed analysis, brainstorming, verification, and refinement of ideas. After “</think>
,” in the Solution section, provide the final, logical, and accurate answer, clearly derived from the exploration in the Thought section. If applicable, include the answer inoxed{} for closed-form results like multiple choices or mathematical solutions. User: This is the problem:
7.177. $\frac{\log _{2}\left(9-2^{x}\right)}{3-x}=1$.

Translate the above text into English, please keep the original text's line breaks and format, and output the translation result directly.

7.177. $\frac{\log